# Explore data from a BGC Argo Float

In [2]:
import numpy as np
from numpy.ma import masked_values as maval
import gsw
import xarray as xr
import pandas as pd
import os.path as op
from datetime import datetime, timedelta
from scipy.interpolate import PchipInterpolator as pchip
from scipy.signal import medfilt
import dask.array as dsar
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib import cm
import matplotlib.colors as clr
import matplotlib.ticker as tick
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import cmocean as cmocean
import os

%matplotlib inline
# not sure if I need to download all these modules for the script  
# below, just copied them over from another script. 

In [3]:
float_id = 12888

this is the "UW float id" for the float that was released with the 
gliders. All the SOCCOM floats (https://soccom.princeton.edu/) can be
found at ( https://www.mbari.org/science/upper-ocean-systems/chemical-sensor-group/soccom-float-visualization/)

In [5]:
# Load Float Data
data_dir = '/Users/dhruvbalwada/OneDrive/sogos_data/'
HRdir = data_dir+"data/raw/SOCCOM/manual_download/SOCCOM_HRQC_LIAR_netcdf_20191201/"
LRdir = data_dir+"data/raw/SOCCOM/manual_download/SOCCOM_LRQC_LIAR_netcdf_20191201/"

float_dic_HR = {}
float_dic_LR = {}

float_ds_HR = xr.open_dataset(HRdir + str(float_id) + "SOOCN_HRQC.nc")
float_ds_LR = xr.open_dataset(LRdir + str(float_id) + "SOOCNQC.nc")
#There is a high resolution and low resolution data set, 
# I believe the difference is in the vertical resolution.

In [6]:
float_ds_HR
# There are a lot of Data variables.

<xarray.Dataset>
Dimensions:              (NPARAMETER: 46, N_LEVELS: 558, N_PROF: 29)
Dimensions without coordinates: NPARAMETER, N_LEVELS, N_PROF
Data variables:
    Cruise               |S11 ...
    Station              (N_PROF) int32 ...
    Lon                  (N_PROF) float64 ...
    Lat                  (N_PROF) float64 ...
    Lat_QF               (N_PROF) |S1 ...
    Lat_QFA              (N_PROF) float64 ...
    Pressure             (N_PROF, N_LEVELS) float64 ...
    Pressure_QF          (N_PROF, N_LEVELS) |S1 ...
    Pressure_QFA         (N_PROF, N_LEVELS) float64 ...
    Temperature          (N_PROF, N_LEVELS) float64 ...
    Temperature_QF       (N_PROF, N_LEVELS) |S1 ...
    Temperature_QFA      (N_PROF, N_LEVELS) float64 ...
    Salinity             (N_PROF, N_LEVELS) float64 ...
    Salinity_QF          (N_PROF, N_LEVELS) |S1 ...
    Salinity_QFA         (N_PROF, N_LEVELS) float64 ...
    Sigma_theta          (N_PROF, N_LEVELS) float64 ...
    Sigma_theta_QF       (N_PROF, N_LEVELS) |S1 ...
    Sigma_theta_QFA      (N_PROF, N_LEVELS) float64 ...
    Depth                (N_PROF, N_LEVELS) float64 ...
    Depth_QF             (N_PROF, N_LEVELS) |S1 ...
    Depth_QFA            (N_PROF, N_LEVELS) float64 ...
    Oxygen               (N_PROF, N_LEVELS) float64 ...
    Oxygen_QF            (N_PROF, N_LEVELS) |S1 ...
    Oxygen_QFA           (N_PROF, N_LEVELS) float64 ...
    OxygenSat            (N_PROF, N_LEVELS) float64 ...
    OxygenSat_QF         (N_PROF, N_LEVELS) |S1 ...
    OxygenSat_QFA        (N_PROF, N_LEVELS) float64 ...
    Nitrate              (N_PROF, N_LEVELS) float64 ...
    Nitrate_QF           (N_PROF, N_LEVELS) |S1 ...
    Nitrate_QFA          (N_PROF, N_LEVELS) float64 ...
    Chl_a                (N_PROF, N_LEVELS) float64 ...
    Chl_a_QF             (N_PROF, N_LEVELS) |S1 ...
    Chl_a_QFA            (N_PROF, N_LEVELS) float64 ...
    Chl_a_corr           (N_PROF, N_LEVELS) float64 ...
    Chl_a_corr_QF        (N_PROF, N_LEVELS) |S1 ...
    Chl_a_corr_QFA       (N_PROF, N_LEVELS) float64 ...
    b_bp700              (N_PROF, N_LEVELS) float64 ...
    b_bp700_QF           (N_PROF, N_LEVELS) |S1 ...
    b_bp700_QFA          (N_PROF, N_LEVELS) float64 ...
    b_bp_corr            (N_PROF, N_LEVELS) float64 ...
    b_bp_corr_QF         (N_PROF, N_LEVELS) |S1 ...
    b_bp_corr_QFA        (N_PROF, N_LEVELS) float64 ...
    POC                  (N_PROF, N_LEVELS) float64 ...
    POC_QF               (N_PROF, N_LEVELS) |S1 ...
    POC_QFA              (N_PROF, N_LEVELS) float64 ...
    pHinsitu             (N_PROF, N_LEVELS) float64 ...
    pHinsitu_QF          (N_PROF, N_LEVELS) |S1 ...
    pHinsitu_QFA         (N_PROF, N_LEVELS) float64 ...
    pH25C                (N_PROF, N_LEVELS) float64 ...
    pH25C_QF             (N_PROF, N_LEVELS) |S1 ...
    pH25C_QFA            (N_PROF, N_LEVELS) float64 ...
    TALK_LIAR            (N_PROF, N_LEVELS) float64 ...
    TALK_LIAR_QF         (N_PROF, N_LEVELS) |S1 ...
    TALK_LIAR_QFA        (N_PROF, N_LEVELS) float64 ...
    DIC_LIAR             (N_PROF, N_LEVELS) float64 ...
    DIC_LIAR_QF          (N_PROF, N_LEVELS) |S1 ...
    DIC_LIAR_QFA         (N_PROF, N_LEVELS) float64 ...
    pCO2_LIAR            (N_PROF, N_LEVELS) float64 ...
    pCO2_LIAR_QF         (N_PROF, N_LEVELS) |S1 ...
    pCO2_LIAR_QFA        (N_PROF, N_LEVELS) float64 ...
    Type                 |S1 ...
    mon_day_yr           (N_PROF) |S10 ...
    hh_mm                (N_PROF) |S5 ...
    Parameters           (NPARAMETER) |S19 ...
    JULD                 (N_PROF) datetime64[ns] ...
    REFERENCE_DATE_TIME  object ...
Attributes:
    Comments:  \n//0\n//File updated on 11/26/2019 20:17\n//!! ADJUSTED DATA ...